In [0]:
deliveries_df = spark.read.option("header", True).csv("/FileStore/tables/deliveries.csv")
deliveries_df.show(5)

+--------+------+--------------------+--------------------+----+----+-----------+-------+-----------+------------+----------+----------+-----------+---------+----------------+--------------+-------+
|match_id|inning|        batting_team|        bowling_team|over|ball|     batter| bowler|non_striker|batsman_runs|extra_runs|total_runs|extras_type|is_wicket|player_dismissed|dismissal_kind|fielder|
+--------+------+--------------------+--------------------+----+----+-----------+-------+-----------+------------+----------+----------+-----------+---------+----------------+--------------+-------+
|  335982|     1|Kolkata Knight Ri...|Royal Challengers...|   0|   1| SC Ganguly|P Kumar|BB McCullum|           0|         1|         1|    legbyes|        0|              NA|            NA|     NA|
|  335982|     1|Kolkata Knight Ri...|Royal Challengers...|   0|   2|BB McCullum|P Kumar| SC Ganguly|           0|         0|         0|       null|        0|              NA|            NA|     NA|
|  33

In [0]:
deliveries_df.write.format('delta').mode('overwrite').save('/FileStore/tables/deliveries_delta')


In [0]:
spark.sql("""
    CREATE TABLE IF NOT EXISTS deliveries_delta_table
    USING DELTA
    LOCATION '/FileStore/tables/deliveries_delta'
""")


Out[15]: DataFrame[]

In [0]:
spark.sql("SHOW TABLES").show()


+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| default|      deliveries_csv|      false|
| default|deliveries_delta_...|      false|
+--------+--------------------+-----------+



In [0]:
from pyspark.sql.functions import lit

updated_df = deliveries_df.withColumn("dummy_column", lit("test_value"))
updated_df.write.format('delta').option("mergeSchema", "true").mode('overwrite').save('/FileStore/tables/deliveries_delta')


In [0]:
spark.sql("SELECT * FROM deliveries_delta_table LIMIT 5").show()


+--------+------+--------------------+--------------------+----+----+-----------+-------+-----------+------------+----------+----------+-----------+---------+----------------+--------------+-------+------------+
|match_id|inning|        batting_team|        bowling_team|over|ball|     batter| bowler|non_striker|batsman_runs|extra_runs|total_runs|extras_type|is_wicket|player_dismissed|dismissal_kind|fielder|dummy_column|
+--------+------+--------------------+--------------------+----+----+-----------+-------+-----------+------------+----------+----------+-----------+---------+----------------+--------------+-------+------------+
|  335982|     1|Kolkata Knight Ri...|Royal Challengers...|   0|   1| SC Ganguly|P Kumar|BB McCullum|           0|         1|         1|    legbyes|        0|              NA|            NA|     NA|  test_value|
|  335982|     1|Kolkata Knight Ri...|Royal Challengers...|   0|   2|BB McCullum|P Kumar| SC Ganguly|           0|         0|         0|       null|    

In [0]:
spark.sql("DESCRIBE HISTORY deliveries_delta_table").show(truncate=False)


+-------+-------------------+----------------+---------------------------+---------+--------------------------------------+----+-----------------+--------------------+-----------+-----------------+-------------+-------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp          |userId          |userName                   |operation|operationParameters                   |job |notebook         |clusterId           |readVersion|isolationLevel   |isBlindAppend|operationMetrics                                                   |userMetadata|engineInfo                         |
+-------+-------------------+----------------+---------------------------+---------+--------------------------------------+----+-----------------+--------------------+-----------+-----------------+-------------+-------------------------------------------------------------------+------------+-----------------------------------+
|3      |2025

In [0]:
# Step 6: Query old version (version 0)
spark.read.format("delta").option("versionAsOf", 0).load("/FileStore/tables/deliveries_delta").show(5)


+--------+------+--------------------+--------------------+----+----+-----------+-------+-----------+------------+----------+----------+-----------+---------+----------------+--------------+-------+
|match_id|inning|        batting_team|        bowling_team|over|ball|     batter| bowler|non_striker|batsman_runs|extra_runs|total_runs|extras_type|is_wicket|player_dismissed|dismissal_kind|fielder|
+--------+------+--------------------+--------------------+----+----+-----------+-------+-----------+------------+----------+----------+-----------+---------+----------------+--------------+-------+
|  335982|     1|Kolkata Knight Ri...|Royal Challengers...|   0|   1| SC Ganguly|P Kumar|BB McCullum|           0|         1|         1|    legbyes|        0|              NA|            NA|     NA|
|  335982|     1|Kolkata Knight Ri...|Royal Challengers...|   0|   2|BB McCullum|P Kumar| SC Ganguly|           0|         0|         0|       null|        0|              NA|            NA|     NA|
|  33